## Fit material properties Vs mc to obtain functions for UMAT

### Imports and working path
Run the following section to import the required libraries and set the working directory. 

In [ ]:
# Imports
from math import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Math
from sklearn.metrics import r2_score
import importlib
import Micromaterial_Calculation as micromat
import os
importlib.reload(micromat)
import mpmath as mp
mp.dps = 150 # increase precision to stabilize matrix operations (default = 15)
np.set_printoptions(suppress=True)

# Set working path
runpath = os.getcwd() + '/'  # main path
print(f"Working on {runpath}")

### Calculate hygroelastic material properties of layers for varying moisture content

In [ ]:
# Set arrays
moisture = [0.12] #np.arange(0, 0.24 + 0.02, 0.02)  # moisture content (from 0 to 0.3)
eng_prop_names = ['E1', 'E2', 'E3', 'v23', 'v13', 'v12', 'G23', 'G13', 'G12']
alpha_names = ['alpha1', 'alpha2', 'alpha3']
prop_names = eng_prop_names + alpha_names
layer_names = ['ML', 'P', 'S1', 'S1-', 'S2', 'S3']
# Calculate engineering constants and hygroexpansion coefficients for each moisture value
eng_const = []
for mc in moisture:
    temp = micromat.layer_mat_prop(mc, runpath, 'mht_full')
    eng_const_temp = [micromat.convert_mat_prop(mat, 'o', 'c', 'e')[0] for mat in temp[0]]
    eng_const.append({'w': mc, 'eng_const': eng_const_temp, 'extra': temp[1]})
# Create dataframe
rows = [{
        "w": entry['w'],
        "layer": layer_names[i] if i < len(layer_names) else f"Layer_{i}",
        **{k: v for k, v in zip(eng_prop_names, props)},
        **{k: v for k, v in zip(alpha_names, entry['extra'][i])}}
    for entry in eng_const
    for i, props in enumerate(entry['eng_const'])
    if not (i < len(layer_names) and layer_names[i] == "S1-")]
df_all = pd.DataFrame(rows)
# Compute averaged properties for ML and P
for mc in df_all['w'].unique():
    for layer in ['ML', 'P']:
        mask = (df_all['w'] == mc) & (df_all['layer'] == layer)
        
        if mask.any():
            # Compute the mean for each group
            E_mean = df_all.loc[mask, ['E1', 'E2', 'E3']].mean(axis=1)
            v_mean = df_all.loc[mask, ['v12', 'v13', 'v23']].mean(axis=1)
            G_mean = df_all.loc[mask, ['G12', 'G13', 'G23']].mean(axis=1)
            
            # Assign the mean to all three corresponding columns
            df_all.loc[mask, ['E1', 'E2', 'E3']] = np.column_stack([E_mean] * 3)
            df_all.loc[mask, ['v12', 'v13', 'v23']] = np.column_stack([v_mean] * 3)
            df_all.loc[mask, ['G12', 'G13', 'G23']] = np.column_stack([G_mean] * 3)

# Display dataframe
display(df_all)

### Fit material properties to moisture

In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")
# Set fontsizes
titlesize = 18
labelsize = 14

def fit_polynomial(x, y, degree):
    # Fit polynomial
    coeffs = np.polyfit(x, y, degree)
    poly_eq = np.poly1d(coeffs)
    fitted_y = poly_eq(x)
    r2 = r2_score(y, fitted_y)

    return coeffs, fitted_y, r2
# end: fit_polynomial

def plot_fit_mat_properties(dataset, titlesize, labelsize, poly_degree_dict):
    layers = dataset['layer'].unique()
    markersize = 2
    for layer in layers:
        df_layer = dataset[dataset['layer'] == layer]
        poly_degree = poly_degree_dict[layer]
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # --- Subplot 1: Engineering Constants ---
        ax1.set_xlabel("mc [-]", fontsize=labelsize)
        ax1.tick_params(labelsize=labelsize)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.set_ylabel(r"$E_{ii}$ [GPa]", color='blue', fontsize=labelsize)

        # Plot Young's moduli (primary y-axis)
        for var, color, label in zip(["E1", "E2", "E3"],
                                     ["blue", "cyan", "navy"],
                                     [r"${E_1}$", r"${E_2}$", r"${E_3}$"]):
            # Plot data
            ax1.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax1.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
        
        # Plot Poisson's ratios and shear moduli (secondary y-axis)
        ax3 = ax1.twinx()
        ax3.set_ylabel(r"$\nu_{ij}$ [-], $10^{-1} \times G_{ij}$ [GPa]", color='red', fontsize=labelsize)

        for var, color, label in zip(["v23", "v13", "v12", "G23", "G13", "G12"],
                                     ["red", "pink", "orange", "purple", "magenta", "brown"],
                                     [r"${\nu_{23}}$", r"${\nu_{13}}$", r"${\nu_{12}}$", r"${G_{23}}$", r"${G_{13}}$", r"${G_{12}}$"]):
            y_data = df_layer[var] / 10 if "G" in var else df_layer[var]
            ax3.plot(df_layer["mc"], y_data, 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            y_fit = y_fit / 10 if "G" in var else y_fit
            ax3.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
            
        ax3.tick_params(axis='y', labelcolor='red', labelsize=labelsize)

        # --- Grid Alignment ---
        ax1.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax3.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax1.set_ylim(bottom=max(0.01, ax1.get_ylim()[0]))
        ax3.set_ylim(bottom=max(0.01, ax3.get_ylim()[0]))
        ax1.set_yticks(np.linspace(ax1.get_ylim()[0], ax1.get_ylim()[1], num=6))
        ax3.set_yticks(np.linspace(ax3.get_ylim()[0], ax3.get_ylim()[1], num=6))
        ax1.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax3.grid(False)
        
        # --- Subplot 2: Hygroexpansion Coefficients ---
        ax2.set_xlabel("mc [-]", fontsize=labelsize)
        ax2.tick_params(labelsize=labelsize)
        ax2.set_ylabel(r"$\alpha_{1}$ [-]", color='green', fontsize=labelsize)

        # Plot alpha1 (primary y-axis)
        ax2.plot(df_layer["mc"], df_layer["alpha1"], 'o', label=r"${\alpha_1}$", color="green", markersize=markersize)
        # Plot polynomial fit
        coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer["alpha1"], poly_degree)
        ax2.plot(df_layer["mc"], y_fit, '-', color=color)
        ax2.plot(df_layer["mc"], y_fit, '-', color="green")
        ax2.tick_params(axis='y', labelcolor='green')

        # Plot alpha2 and alpha3 (secondary y-axis)
        ax4 = ax2.twinx()
        for var, color, label in zip(["alpha2", "alpha3"], 
                                     ["olive", "darkgreen"], 
                                     [r"${\alpha_2}$", r"${\alpha_3}$"]):
            ax4.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax4.plot(df_layer["mc"], y_fit, '-', color=color)
            
        ax4.tick_params(axis='y', labelcolor='olive', labelsize=labelsize)

        # --- Grid Alignment ---
        ax2.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax2.set_ylim(0, 0.2)
        ax4.set_ylim(0.2, 0.5)
        ax2.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax4.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax2.set_yticks(np.linspace(ax2.get_ylim()[0], ax2.get_ylim()[1], num=6))
        ax4.set_yticks(np.linspace(ax4.get_ylim()[0], ax4.get_ylim()[1], num=6))
        ax2.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax4.grid(False)

        # --- Legends ---
        handles_ec, labels_ec = ax1.get_legend_handles_labels()
        handles_vg, labels_vg = ax3.get_legend_handles_labels()
        ax1.legend(handles=handles_ec + handles_vg, labels=labels_ec + labels_vg, loc="upper right", fontsize=labelsize)

        handles_alpha, labels_alpha = ax2.get_legend_handles_labels()
        handles_alpha2_3, labels_alpha2_3 = ax4.get_legend_handles_labels()
        ax2.legend(handles=handles_alpha + handles_alpha2_3, labels=labels_alpha + labels_alpha2_3, loc="upper right", fontsize=labelsize)

        fig.suptitle(f"{layer}", fontsize=titlesize, fontweight='bold')
        plt.tight_layout()
    
    return [ax1, ax3, ax2, ax4]
# end: plot_fit_mat_properties

# Plot material properties and fitting
CML_degree = 10
S_degree = CML_degree
poly_degree_dict = {
    "ML": CML_degree,
    "P": CML_degree,
    "S1": S_degree,
    "S2": S_degree,
    "S3": S_degree
}

# Check number of unique moisture values
unique_mc = df_all['w'].unique()
if len(unique_mc) < 2:
    print("Only one moisture content value (w={}) in the dataset. No fitting plot will be generated.".format(unique_mc[0]))
else:
    plot_fit_mat_properties(df_all, titlesize, labelsize, poly_degree_dict)
    plt.show()

### Calculate equivalent properties of CML and S for varying moisture content

In [ ]:
MFA = [0., 0., 60., -60., 15., 75.]
tissues = ["EW", "TW", "LW"]
t_tot = {
    "EW": 2.55,
    "TW": 3.85,
    "LW": 6.}

t_0 = [0.175, 0.175, 0.125, 0.125, 0, 0.035]

t_S2 = {
    "EW": 0,
    "TW": 0,
    "LW": 0}

records = []
for tissue in tissues:
    t_S2[tissue] = round((t_tot[tissue] - sum(t_0)),3)
    t = [0.175, 0.175, 0.125, 0.125, t_S2[tissue], 0.035]
    for mc in moisture:
        # Calculate layers stiffness
        C = micromat.layer_mat_prop(mc, runpath, 'mht_full')[0]
        # Compute CML (matrix)
        eng_cml = np.array(micromat.laminate_3D_mat_prop(MFA[:2], t[:2], C[:2]))
        E_raw = eng_cml[0:3].mean()
        nu_raw = eng_cml[3:6].mean()
        G_raw = eng_cml[6:9].mean()
        eng_cml[0:3] = E_raw
        eng_cml[3:6] = nu_raw
        eng_cml[6:9] = G_raw
        # Compute S (fiber)
        eng_s = micromat.laminate_3D_mat_prop(MFA[2:], t[2:], C[2:])
        C_eq = micromat.full_laminate_matrix(MFA[2:], t[2:], C[2:])
        #display(C_eq)
        eng_s = np.array(eng_s)

        # Store data
        for layer_name, eng_vals in (('MATRIX', eng_cml), (tissue, eng_s)):
            rec = {'w': mc, 'layer': layer_name,}
            rec.update({name: float(val)
                        for name, val in zip(eng_prop_names, eng_vals)})
            records.append(rec)

# Create dataframe
df_eq = pd.DataFrame(records)
df_eq = df_eq[['w', 'layer'] + eng_prop_names]
df_eq.drop_duplicates(inplace=True)

display(df_eq)

### Fit material properties to moisture content

In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")
# Set fontsizes
titlesize = 18
labelsize = 14

def fit_polynomial(x, y, degree):
    # Fit polynomial
    coeffs = np.polyfit(x, y, degree)
    poly_eq = np.poly1d(coeffs)
    fitted_y = poly_eq(x)
    r2 = r2_score(y, fitted_y)

    return coeffs, fitted_y, r2
# end: fit_polynomial

def plot_fit_mat_properties(dataset, titlesize, labelsize, poly_degree_dict):
    layers = dataset['layer'].unique()
    markersize = 2
    for layer in layers:
        df_layer = dataset[dataset['layer'] == layer]
        poly_degree = poly_degree_dict[layer]
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # --- Subplot 1: Engineering Constants ---
        ax1.set_xlabel("mc [-]", fontsize=labelsize)
        ax1.tick_params(labelsize=labelsize)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.set_ylabel(r"$E_{ii}$ [GPa]", color='blue', fontsize=labelsize)

        # Plot Young's moduli (primary y-axis)
        for var, color, label in zip(["E1", "E2", "E3"],
                                     ["blue", "cyan", "navy"],
                                     [r"${E_1}$", r"${E_2}$", r"${E_3}$"]):
            # Plot data
            ax1.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax1.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
        
        # Plot Poisson's ratios and shear moduli (secondary y-axis)
        ax3 = ax1.twinx()
        ax3.set_ylabel(r"$\nu_{ij}$ [-], $10^{-1} \times G_{ij}$ [GPa]", color='red', fontsize=labelsize)

        for var, color, label in zip(["v23", "v13", "v12", "G23", "G13", "G12"],
                                     ["red", "pink", "orange", "purple", "magenta", "brown"],
                                     [r"${\nu_{23}}$", r"${\nu_{13}}$", r"${\nu_{12}}$", r"${G_{23}}$", r"${G_{13}}$", r"${G_{12}}$"]):
            y_data = df_layer[var] / 10 if "G" in var else df_layer[var]
            ax3.plot(df_layer["mc"], y_data, 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            y_fit = y_fit / 10 if "G" in var else y_fit
            ax3.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
            
        ax3.tick_params(axis='y', labelcolor='red', labelsize=labelsize)

        # --- Grid Alignment ---
        ax1.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax3.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax1.set_ylim(bottom=max(0.01, ax1.get_ylim()[0]))
        ax3.set_ylim(bottom=max(0.01, ax3.get_ylim()[0]))
        ax1.set_yticks(np.linspace(ax1.get_ylim()[0], ax1.get_ylim()[1], num=6))
        ax3.set_yticks(np.linspace(ax3.get_ylim()[0], ax3.get_ylim()[1], num=6))
        ax1.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax3.grid(False)
        
        # --- Subplot 2: Hygroexpansion Coefficients ---
        ax2.set_xlabel("mc [-]", fontsize=labelsize)
        ax2.tick_params(labelsize=labelsize)
        ax2.set_ylabel(r"$\alpha_{1}$ [-]", color='green', fontsize=labelsize)

        # Plot alpha1 (primary y-axis)
        ax2.plot(df_layer["mc"], df_layer["alpha1"], 'o', label=r"${\alpha_1}$", color="green", markersize=markersize)
        # Plot polynomial fit
        coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer["alpha1"], poly_degree)
        ax2.plot(df_layer["mc"], y_fit, '-', color=color)
        ax2.plot(df_layer["mc"], y_fit, '-', color="green")
        ax2.tick_params(axis='y', labelcolor='green')

        # Plot alpha2 and alpha3 (secondary y-axis)
        ax4 = ax2.twinx()
        for var, color, label in zip(["alpha2", "alpha3"], 
                                     ["olive", "darkgreen"], 
                                     [r"${\alpha_2}$", r"${\alpha_3}$"]):
            ax4.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax4.plot(df_layer["mc"], y_fit, '-', color=color)
            
        ax4.tick_params(axis='y', labelcolor='olive', labelsize=labelsize)

        # --- Grid Alignment ---
        ax2.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax2.set_ylim(0, 0.2)
        ax4.set_ylim(0.2, 0.5)
        ax2.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax4.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax2.set_yticks(np.linspace(ax2.get_ylim()[0], ax2.get_ylim()[1], num=6))
        ax4.set_yticks(np.linspace(ax4.get_ylim()[0], ax4.get_ylim()[1], num=6))
        ax2.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax4.grid(False)

        # --- Legends ---
        handles_ec, labels_ec = ax1.get_legend_handles_labels()
        handles_vg, labels_vg = ax3.get_legend_handles_labels()
        ax1.legend(handles=handles_ec + handles_vg, labels=labels_ec + labels_vg, loc="upper right", fontsize=labelsize)

        handles_alpha, labels_alpha = ax2.get_legend_handles_labels()
        handles_alpha2_3, labels_alpha2_3 = ax4.get_legend_handles_labels()
        ax2.legend(handles=handles_alpha + handles_alpha2_3, labels=labels_alpha + labels_alpha2_3, loc="upper right", fontsize=labelsize)

        fig.suptitle(f"{layer}", fontsize=titlesize, fontweight='bold')
        plt.tight_layout()
    
    return [ax1, ax3, ax2, ax4]
# end: plot_fit_mat_properties

# Plot material properties and fitting
poly_deg = 10
poly_degree_dict = {"MATRIX": poly_deg, "EW": poly_deg, "TW": poly_deg, "LW": poly_deg}

# Check number of unique moisture values
unique_mc = df_all['w'].unique()
if len(unique_mc) < 2:
    print("Only one moisture content value (w={}) in the dataset. No fitting plot will be generated.".format(unique_mc[0]))
else:
    plot_fit_mat_properties(df_eq, titlesize, labelsize, poly_degree_dict)
    plt.show()


### Save material properties in inc files for Fortran routine

In [ ]:
# Function to write a single Fortran file with engineering constants
def write_eng_const_file(filepath, dataset, poly_degree_dict):

    properties = [('E1', 'E1'), ('E2', 'E2'), ('E3', 'E3'),
                  ('v23', 'nu23'), ('v13', 'nu13'), ('v12', 'nu12'),
                  ('G23', 'G23'), ('G13', 'G13'), ('G12', 'G12')]

    layers = dataset['layer'].unique()
    filename = filepath + "/tracheids_eng_const.inc"

    with open(filename, 'w') as f:

        # Fixed-form Fortran comment
        f.write("C Engineering constants for tracheids and matrix\n")

        for idx, layer in enumerate(layers):

            if idx == 0:
                f.write("      IF (MATNAME == '{}') THEN\n".format(layer))
            else:
                f.write("      ELSE IF (MATNAME == '{}') THEN\n".format(layer))

            df_layer = dataset[dataset['layer'] == layer]

            for var, label in properties:
                unique_w = df_layer['w'].unique()

                if len(unique_w) > 1:
                    # Polynomial fit → evaluate at current KW_MC symbolically
                    poly_degree = poly_degree_dict[layer]
                    coeffs, _, _ = fit_polynomial(
                        df_layer['w'], df_layer[var], poly_degree
                    )

                    expr = ""
                    for i, c in enumerate(coeffs):
                        deg = poly_degree - i
                        if deg == 0:
                            expr += "({:.3f}D0)".format(c)
                        elif deg == 1:
                            expr += "({:.3f}D0)*(KW_MC)".format(c)
                        else:
                            expr += "({:.3f}D0)*(KW_MC)**{}".format(c, deg)
                        if i < len(coeffs) - 1:
                            expr += " + "

                    f.write("      KW_{} = {}\n".format(label, expr))

                else:
                    const_value = df_layer[var].iloc[0]
                    f.write(
                        "      KW_{} = ({:.3f}D0)\n".format(label, const_value)
                    )

        # Final ELSE block
        f.write("      ELSE\n")
        f.write("      PRINT *, 'Error: Material ', MATNAME, ' not supported.'\n")
        f.write("      RETURN\n")
        f.write("      END IF\n")

    return
# end: write_eng_const_file


# Generate files for engineering constants
# Save in Elastic_Test folder
savepath = os.path.dirname(os.getcwd()) + "/Modules" 
write_eng_const_file(savepath, df_eq, poly_degree_dict)
print(f"Files generated in: {savepath}")

# Save in Original_Folder for creep simulations
savepath = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "Original_Folder", "Modules")
if not os.path.exists(savepath): os.makedirs(savepath)
write_eng_const_file(savepath, df_eq, poly_degree_dict)
print(f"Files generated in: {savepath}")
